# Deep Learning

This notebook only handles the training part

In [1]:
%matplotlib inline

import cPickle as pickle
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import math

### Hyperparameter

In [2]:
basket_static ='/mnt/basket/processingData/data/notMNIST'
basket_volatile = "/mnt/basket/processingData/volatile/notMNIST"
raw_data_pickle_filename = "notMNIST.pickle"
training_results_pickle_filename = "2_fullyconnected_separeated__training_results.pickle"

### Load the data we generated in `01_NormalizeImage.ipynb`

In [3]:
pickle_file =  os.path.join(basket_volatile,raw_data_pickle_filename)

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  del save  # hint to help gc free up memory
  print 'Training set', train_dataset.shape, train_labels.shape

Training set (476202, 28, 28) (476202,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix
- labels as float 1-hot encodings

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
print 'Training set', train_dataset.shape, train_labels.shape

Training set (476202, 784) (476202, 10)


### Utility function

In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

### Shuffle the training data

In [6]:
def randomize(dataset, labels):
  permutation = np.random.permutation(labels.shape[0])
  shuffled_dataset = dataset[permutation,:]
  shuffled_labels = labels[permutation]
  return shuffled_dataset, shuffled_labels

## S.D.G. with one hidden layer

### Training

##### Define the graph

In [7]:
batch_size = 128
#old 0.5
learning_rate = 0.5
graph = tf.Graph()

with graph.as_default():

  #Input data. For the training data, we use a placeholder that will be fed at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size)) #(128,784)
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels)) #(128,10)
    
  #Define HyperParameters (NN structuring constants)
  inputLayerSize = image_size * image_size # inputLayerSize is always equal to width of X-ndarray (200000,784)
  hiddenLayerSize = 1024
  outputLayerSize = 10
    
  # Variables:
  ## hidden layer
  weights_hidden_1 = tf.Variable(tf.truncated_normal([inputLayerSize, hiddenLayerSize])) #(784,1024)
  biases_hidden_1 = tf.Variable(tf.zeros([hiddenLayerSize])) #(1024)
  ## linear layer
  weights_linear = tf.Variable(tf.truncated_normal([hiddenLayerSize, outputLayerSize])) #(1024,10)
  biases_linear = tf.Variable(tf.zeros([outputLayerSize])) #(10)

  # Hidden layer (RELU)
  train_hiddenLayer = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden_1) + biases_hidden_1) #(128,1024)

  # linear  layer
  ## a) logits  (tune: weights_linear,biases_linear)
  train_logits = tf.matmul(train_hiddenLayer, weights_linear) + biases_linear  #(128,10) => (batch_size, num_labels)
  ## b) loss
  train_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(train_logits, tf_train_labels))#([])
  # c) Optimizer ( Creates an optimizer and applies the gradients to all trainable variables. )
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(train_loss)
 
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(train_logits)

  print tf.Tensor.get_shape(train_hiddenLayer) #static shape

TensorShape([Dimension(128), Dimension(1024)])


##### Run the graph

http://neuralnetworksanddeeplearning.com/chap1.html
In each epoch, it starts by randomly shuffling the training data and then partition it into mini-batches of the appropriate size. This is an easy way of sampling randomly from the training data. <br>
Then for each mini_batch we apply a single step of gradient descent, which updates the network weights and biases.

In [8]:
print "size training data : " , train_labels.shape[0]

# The entire training dataset going through the entire network once. Related to overfitting.
epoch = 3

plt.clf()

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized, start training"
  for epoch in xrange(0,epoch):
    print "Epoch ", epoch + 1, "started"
    # Randomize training dataset
    train_dataset, train_labels = randomize(train_dataset, train_labels)
    # calculate number of iterations needed to go through the full training data once
    num_steps = train_labels.shape[0] / batch_size
    for step in xrange(num_steps):
        offset = (step * batch_size) %  (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, loss, predictions = session.run([optimizer, train_loss, train_prediction], feed_dict=feed_dict)

        # Only print a summary, else the output is to long
        if (offset % (batch_size*500) == 0):
            print "   Training loss summary for step " , step, " : ", loss
    print "Training accuracy: %.1f%%" % accuracy(predictions, batch_labels)
  w1, b1, w2, b2 = session.run([weights_hidden_1, biases_hidden_1, weights_linear, biases_linear])

size training data :  476202
Initialized, start training
Epoch  1 started
   Training loss summary for step  0  :  513.386
   Training loss summary for step  500  :  12.0766
   Training loss summary for step  1000  :  6.20933
   Training loss summary for step  1500  :  4.93562
   Training loss summary for step  2000  :  9.2379
   Training loss summary for step  2500  :  4.70921
   Training loss summary for step  3000  :  4.09972
   Training loss summary for step  3500  :  3.38553
Training accuracy: 80.5%
Epoch  2 started
   Training loss summary for step  0  :  5.45085
   Training loss summary for step  500  :  3.83473
   Training loss summary for step  1000  :  2.86659
   Training loss summary for step  1500  :  3.1793
   Training loss summary for step  2000  :  0.735756
   Training loss summary for step  2500  :  5.13519
   Training loss summary for step  3000  :  3.34327
   Training loss summary for step  3500  :  1.84174
Training accuracy: 83.6%
Epoch  3 started
   Training loss su

#### Plot training loss

In [9]:
#batch_sizes = np.arange(0, 1280, 128)
#print training_summary
#plt.bar(training_summary[:,1], training_summary, align='center', alpha=0.5)
#plt.show()
#del classBalance

#### Serialize training weights on disk

In [10]:
pickle_file = os.path.join(basket_volatile,training_results_pickle_filename)

try:
  f = open(pickle_file, 'wb')
  save = {
    'weights_hidden_1': w1,
    'biases_hidden_1': b1,
    'weights_linear': w2,
    'biases_linear': b2,
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print 'Unable to save data to', pickle_file, ':', e
  raise